In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
import pickle

In [2]:
data = pd.read_csv('Churn_Modelling.csv')

data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [4]:
LabelEncoder_Gen = LabelEncoder()
data['Gender'] = LabelEncoder_Gen.fit_transform(data['Gender'])

In [5]:
# OneHotEncode geo column

from sklearn.preprocessing import OneHotEncoder
OneHotEnc_Geo = OneHotEncoder(sparse_output=False)
geo_encoded = OneHotEnc_Geo.fit_transform(data[['Geography']])

In [6]:
col = OneHotEnc_Geo.get_feature_names_out()
col

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
geo_encoded_df = pd.DataFrame(geo_encoded, columns=col)

In [8]:
data  = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [9]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
with open("artifect/level_encoder_gender.pkl",'wb') as file:
    pickle.dump(LabelEncoder_Gen, file)
    
    
with open("artifect/onehot_encoder_geo.pkl",'wb') as file:
    pickle.dump(OneHotEnc_Geo, file)

In [11]:
# devide the feature as independent and dependent features
X = data.drop('Exited', axis=1)
y = data['Exited']

In [12]:
# devide the dataset into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
# scale the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
# save the scaler as pkl 
with open("artifect/scaler.pkl","wb") as file:
    pickle.dump(scaler,file)

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

2025-12-27 22:49:14.252817: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
# builed ANN model 
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(1,activation='sigmoid')
])



In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                416       
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 961 (3.75 KB)
Trainable params: 961 (3.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [19]:
# compile the model
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [20]:
# setup TensorBoard

log_dir = 'log/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)


In [21]:
# setup EarlyStopping callback

EarlyStopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [22]:
hestory = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=69,
                    
                    callbacks=[tensorflow_callback,EarlyStopping_callback])

Epoch 1/69
189/189 [==============================] - 2s 6ms/step - loss: 0.4102 - accuracy: 0.8268 - val_loss: 0.3752 - val_accuracy: 0.8482
Epoch 2/69
189/189 [==============================] - 1s 4ms/step - loss: 0.3554 - accuracy: 0.8543 - val_loss: 0.3554 - val_accuracy: 0.8522
Epoch 3/69
189/189 [==============================] - 1s 4ms/step - loss: 0.3462 - accuracy: 0.8596 - val_loss: 0.3541 - val_accuracy: 0.8582
Epoch 4/69
189/189 [==============================] - 1s 3ms/step - loss: 0.3402 - accuracy: 0.8612 - val_loss: 0.3531 - val_accuracy: 0.8602
Epoch 5/69
189/189 [==============================] - 1s 4ms/step - loss: 0.3368 - accuracy: 0.8597 - val_loss: 0.3489 - val_accuracy: 0.8575
Epoch 6/69
189/189 [==============================] - 1s 3ms/step - loss: 0.3318 - accuracy: 0.8641 - val_loss: 0.3546 - val_accuracy: 0.8555
Epoch 7/69
189/189 [==============================] - 0s 2ms/step - loss: 0.3296 - accuracy: 0.8627 - val_loss: 0.3472 - val_accuracy: 0.8575
Epoch 

In [23]:
model.save('artifect/model.h5')

/Users/apple/Desktop/DeepLearning_Project/venv/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [24]:
%load_ext tensorboard

In [27]:
%tensorboard --logdir log/fit/20251226-224239


Reusing TensorBoard on port 6006 (pid 753), started 0:00:05 ago. (Use '!kill 753' to kill it.)

In [26]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
